In [1]:
!pip3 install streamlit -q
!pip3 install transformers



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_path = f"Afia-manubea/FineTuned-BertTweet-Classification-Model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
from transformers import pipeline
classifier = pipeline("text-classification", model= f"Afia-manubea/FineTuned-BertTweet-Classification-Model")

In [6]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [7]:
import pandas as pd
# Input preprocessing

text = "This covid came with its own agenda"
#text = pd.read_csv("/content/drive/MyDrive/p5/p5 dataset/test_data.csv")
text = preprocess(text)

In [8]:
# PyTorch-based models
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# TensorFlow-based models
# model = TFAutoModelForSequenceClassification.from_pretrained(model_path)
# model.save_pretrained(model_path)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

In [9]:
encoded_input

{'input_ids': tensor([[  101,  2023,  2522, 17258,  2234,  2007,  2049,  2219, 11376,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.6851,  2.0654, -1.0260]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
# score without softmax function

output[0][0].detach().numpy()

array([-1.6850501,  2.0653648, -1.0260166], dtype=float32)

In [12]:
#display score with softmax function

scores

array([0.02199173, 0.9355    , 0.04250831], dtype=float32)

In [13]:
config.id2label = {0: 'NEGATIVE', 1: 'NEUTRAL', 2: 'POSITIVE'}

In [14]:
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
print (f"Classified text:{text}")
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Classified text:This covid came with its own agenda
1) NEUTRAL 0.9355
2) POSITIVE 0.0425
3) NEGATIVE 0.022


# **STREAMLIT APP**

To create a streamlit file, we will write a cell python code into an app.py file using a line magic command. Also we would used local tunnel library to be able to push our work from google colab directly to the web.

###**What is local tunnel lib?**

A **local tunnel** in the context of an npm (Node Package Manager) library usually refers to a tool or library that allows you to expose a locally hosted web application or service to the public internet. These tools are particularly useful during development and testing when you want to share your work with others or test it on various devices.

In [15]:
%%writefile App.py
import time
import pandas as pd
from PIL import Image

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import streamlit as st
import torch

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Afia-manubea/FineTuned-DistilBert-Model")
model = AutoModelForSequenceClassification.from_pretrained("Afia-manubea/FineTuned-DistilBert-Model")

def main():
    # Setup web page
    st.set_page_config(

     page_title="Covid Vaccine tweets Sentiment",
     #page_icon=('nav_pg.jpg'),
     layout="wide",
     menu_items={
         'About': "The source code for this application can be accessed on GitHub "
     }
)
    st.markdown("""
    <style type="text/css">
    blockquote {
        margin: 1em 0px 1em -1px;
        padding: 0px 0px 0px 1.2em;
        font-size: 20px;
        border-left: 5px solid rgb(230, 234, 241);
        # background-color: rgb(129, 164, 182);
    }
    blockquote p {
        font-size: 30px;
        color: #FFFFFF;
    }
    [data-testid=stSidebar] {
        background-color: rgb(129, 164, 182);
        color: #FFFFFF;
    }
    [aria-selected="true"] {
         color: #000000;
    }
    </style>
""", unsafe_allow_html=True)



    st.sidebar.title(' **Covid Vaccine tweets Sentiment**')
    st.sidebar.title(' **Navigation**')

    # Customize the width of the sidebar
    # st.sidebar.set_width(300)

    # Change the background color of the sidebar
    st.sidebar.markdown('<style>body {background-color: #f8f9fa;}</style>', unsafe_allow_html=True)

    page = st.sidebar.radio('Select a Page', ['Home', 'Predict', 'About'])

    if page == 'Home':
        home()

    elif page == 'Predict':
        predict_sentiment()

    elif page == 'About':
        about_info()

def home():



    # Add styled text using CSS
    st.markdown('<p style="color: blue; font-size: 20px;"></p>', unsafe_allow_html=True)
    st.markdown("## FINETUNED BERT SENTIMENT MODEL FOR CATEGORIZATION OF COVID TWEETS")

     # Open and display the image
    img = Image.open('/content/drive/MyDrive/p5/pics/emotional-feedback-concept-illustration (3).zip (Unzipped Files)/9019830.jpg')
    st.image(img)

    st.write("""
     ### To ensure the best of **Experience** on the covid tweet sentiment App ,
    Navigate The App using the sidebar to:

    1. Predict sentiment on tweets
    2. Learn more about this project
    """)


def predict_sentiment():
    st.title("Classify Sentiment")

    # Add an information message
    st.info("Covid Tweets machine learning model to assess if a Twitter post related to vaccinations is positive, neutral, or negative. \nExample: 'Vaccine Who!, and where' ")

    # Getting user input
    tweet_text = st.text_input("Enter the COVID tweet:")

    if st.button("Predict"):
        if tweet_text:
            # Tokenize the input text
            inputs = tokenizer.encode_plus(tweet_text, return_tensors="pt", truncation=True, padding=True)

            # Forward pass through the model
            with torch.no_grad():
                output = model(**inputs)

            # Extract the predicted probabilities
            scores = torch.nn.functional.softmax(output.logits, dim=1).squeeze().tolist()

            # Define the sentiment labels
            labels = ["Negative", "Neutral", "Positive"]

            # Create a dictionary of sentiment scores
            scores_dict = {label: score for label, score in zip(labels, scores)}

            # Display predicted sales
            st.balloons()
            st.progress(10)
            with st.spinner('Wait for it...'):
              time.sleep(10)

            # Display results in a table
            st.write("Sentiment Scores:")
            df = pd.DataFrame(scores_dict.items(), columns=['Sentiment', 'Score'])
            st.table(df)


            # Get the index of the highest score
            max_index = scores.index(max(scores))

            # Get the corresponding sentiment label
            predicted_label = labels[max_index]

            # Display the predicted sentiment label
            st.success(f"Predicted Sentiment: {predicted_label}", icon="✅")

        else:
            st.warning("Please enter a COVID tweet to predict sentiment.")

def about_info():
    st.title("About")

     # Open and display the image
    img = Image.open('/content/drive/MyDrive/p5/pics/emotional-feedback-concept-illustration (1).zip (Unzipped Files)/8447107.jpg')
    st.image(img)

    st.write("""
    Welcome to our Streamlit app! We are dedicated to providing a powerful solution that aims to help governments and public health actors monitor public sentiment towards COVID-19 vaccinations.
    By leveraging the capabilities of a fine-tuned BERT base uncased model from Hugging Face, we have developed a sentiment analysis tool specifically trained on COVID-19 vaccine tweets.

    Our platform allows users to enter a tweet and obtain an accurate sentiment label determined by our model. The sentiment labels include negative, neutral, or positive,
    providing valuable insights into public opinion surrounding COVID-19 vaccination efforts.

    The significance of monitoring public sentiment towards COVID-19 vaccines cannot be overstated. It plays a crucial role in shaping public health policies, vaccine communication strategies,
    and vaccination programs worldwide. By understanding the sentiments expressed in tweets related to COVID-19 vaccines, governments and public health actors can gain valuable
    insights into public perception, concerns, and attitudes.

    Developed by Team Charleston.
    More details about the project and methodology can be found [here](your_link).
    """)


if __name__ == '__main__':
    main()


Writing App.py


In [16]:
!wget -q -O - ipv4.icanhazip.com

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
!streamlit run /content/App.py & npx localtunnel --port 8501

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: /content/App.py
'npx' is not recognized as an internal or external command,
operable program or batch file.
